In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

You should consider upgrading via the 'c:\users\narayan jha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\narayan jha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [2]:
# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

# FindSpark simplifies the process of using Apache Spark with Python

import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import StandardScaler

In [57]:
spark = SparkSession.builder.appName("Practice Project").getOrCreate()

In [7]:
!pip install wget

You should consider upgrading via the 'c:\users\narayan jha\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.



Using legacy 'setup.py install' for wget, since package 'wheel' is not installed.
    Running setup.py install for wget: started
    Running setup.py install for wget: finished with status 'done'


In [9]:
import pandas as pd

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv"
df = pd.read_csv(url)

# Display the first 5 rows
0df.head()


,MPG,Cylinders,Engine Disp,Horsepower,Weight,Accelerate,Year,Origin
0,46.6,4.0,86.0,65.0,2110.0,17.9,80,Japanese
1,44.6,4.0,91.0,67.0,1850.0,13.8,80,Japanese
2,44.3,4.0,90.0,48.0,2085.0,21.7,80,European
3,44.0,4.0,97.0,52.0,2130.0,24.6,82,European
4,43.4,4.0,90.0,48.0,2335.0,23.7,80,European


In [5]:
!curl -o mpg-raw.csv https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg-raw.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 14354  100 14354    0     0   8028      0  0:00:01  0:00:01 --:--:--  8041


In [26]:
df = spark.read.csv("mpg-raw.csv", header=True, inferSchema=True)
df

DataFrame[MPG: double, Cylinders: int, Engine Disp: double, Horsepower: int, Weight: int, Accelerate: double, Year: int, Origin: string]

In [27]:
df.groupBy('Origin').count().orderBy('count').show()

+--------+-----+
|  Origin|count|
+--------+-----+
|    null|    1|
|European|   70|
|Japanese|   88|
|American|  247|
+--------+-----+



In [28]:
df = df.dropDuplicates()
df = df.dropna()
df.show()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|24.0|        4|      134.0|        96|  2702|      13.5|  75|Japanese|
|18.0|        6|      250.0|        88|  3139|      14.5|  71|American|
|29.0|        4|       68.0|        49|  1867|      19.5|  73|European|
|22.4|        6|      231.0|       110|  3415|      15.8|  81|American|
|20.5|        6|      231.0|       105|  3425|      16.9|  77|American|
|20.2|        6|      200.0|        88|  3060|      17.1|  81|American|
|15.0|        8|      429.0|       198|  4341|      10.0|  70|American|
|13.0|        8|      350.0|       175|  4100|      13.0|  73|American|
|18.0|        3|       70.0|        90|  2124|      13.5|  73|Japanese|
|15.5|        8|      350.0|       170|  4165|      11.4|  77|American|
|25.0|        6|      181.0|       110|  2945|      16.4|  82|Am

In [22]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [29]:
df = df.withColumnRenamed("Engine Disp","Engine_Disp")
df.show()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|24.0|        4|      134.0|        96|  2702|      13.5|  75|Japanese|
|18.0|        6|      250.0|        88|  3139|      14.5|  71|American|
|29.0|        4|       68.0|        49|  1867|      19.5|  73|European|
|22.4|        6|      231.0|       110|  3415|      15.8|  81|American|
|20.5|        6|      231.0|       105|  3425|      16.9|  77|American|
|20.2|        6|      200.0|        88|  3060|      17.1|  81|American|
|15.0|        8|      429.0|       198|  4341|      10.0|  70|American|
|13.0|        8|      350.0|       175|  4100|      13.0|  73|American|
|18.0|        3|       70.0|        90|  2124|      13.5|  73|Japanese|
|15.5|        8|      350.0|       170|  4165|      11.4|  77|American|
|25.0|        6|      181.0|       110|  2945|      16.4|  82|Am

In [30]:
df.write.mode("overwrite").parquet("mpg-cleaned.parquet")

In [58]:
# Load the Parquet file into a DataFrame
df = spark.read.parquet("mpg-cleaned.parquet")

# Show the first few rows
df.show()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine_Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|32.2|        4|      108.0|        75|  2265|      15.2|  80|Japanese|
|28.0|        4|      107.0|        86|  2464|      15.5|  76|European|
|26.0|        4|      156.0|        92|  2585|      14.5|  82|American|
|25.0|        4|      104.0|        95|  2375|      17.5|  70|European|
|25.0|        4|      140.0|        75|  2542|      17.0|  74|American|
|19.0|        4|      120.0|        88|  3270|      21.9|  76|European|
|16.5|        8|      350.0|       180|  4380|      12.1|  76|American|
|34.0|        4|      108.0|        70|  2245|      16.9|  82|Japanese|
|29.0|        4|       97.0|        78|  1940|      14.5|  77|European|
|18.6|        6|      225.0|       110|  3620|      18.7|  78|American|
|13.0|        8|      350.0|       155|  4502|      13.5|  72|Am

In [59]:
#Define the StringIndexer pipeline stage
indexer = StringIndexer(inputCol="Origin", outputCol="OriginIndex")

In [60]:
#Define the VectorAssembler pipeline stage
assembler = VectorAssembler(inputCols=['Cylinders','Engine_Disp','Horsepower','Weight','Accelerate','Year'], outputCol="features")

In [61]:
#Define the StandardScaler pipeline stage
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures")

In [62]:
#Define the Model creation pipeline stage
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="MPG")

In [63]:
#Build the pipeline
pipeline = Pipeline(stages=[indexer,assembler, scaler, lr])

In [64]:
#Split the data
(trainingData, testingData) = df.randomSplit([0.7, 0.3], seed=42)

In [65]:
# Fit the pipeline
pipelineModel = pipeline.fit(trainingData)

In [66]:
#Model Evaluation
predictions = pipelineModel.transform(testingData)

In [67]:
#Print the MSE
from pyspark.ml.evaluation import RegressionEvaluator

evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mse")
mse = evaluator.evaluate(predictions)
print(mse)

12.979280116908738


In [68]:
#Print the MAE
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="mae")
mae = evaluator.evaluate(predictions)
print(mae)

2.6320277252905653


In [69]:
#Print the R-Squared(R2)
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="MPG", metricName="r2")
r2 = evaluator.evaluate(predictions)
print(r2)

0.80742616257041


In [70]:
#Model persistance
pipelineModel.write().overwrite().save("Practice_Project")

In [75]:
import os
print(os.path.exists("C:\\Users\\NARAYAN JHA\\AppData\\Local\\Programs\\Python\\Python39\\Scripts\\Practice_Project"))

True


In [85]:
print(spark.sparkContext.getConf().getAll())

[('spark.app.startTime', '1743322028520'), ('spark.app.name', 'Practice Project'), ('spark.rdd.compress', 'True'), ('spark.driver.port', '49917'), ('spark.serializer.objectStreamReset', '100'), ('spark.master', 'local[*]'), ('spark.submit.pyFiles', ''), ('spark.executor.id', 'driver'), ('spark.submit.deployMode', 'client'), ('spark.driver.host', 'LAPTOP-37QNABVB'), ('spark.app.id', 'local-1743322031099'), ('spark.sql.warehouse.dir', 'file:/C:/Users/NARAYAN%20JHA/AppData/Local/Programs/Python/Python39/Scripts/spark-warehouse'), ('spark.ui.showConsoleProgress', 'true')]
